In [1]:
# chainer用モジュールのインポート
import numpy as np
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import  Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
# 入力3つ
x1 = Variable(np.array([1], dtype=np.float32))
x2 = Variable(np.array([2], dtype=np.float32))
x3 = Variable(np.array([3], dtype=np.float32))


In [3]:
z = (x1 - 2 * x2 -1 ) **2 + (x2 * x3 - 1)** 2 + 1

In [4]:
# 順方向に計算
z.data

array([ 42.], dtype=float32)

In [5]:
# 逆伝搬で微分値を取得する
z.backward()

In [6]:
# 偏微分の値を取得する
print(x1.grad)
print(x2.grad)
print(x3.grad)

[-8.]
[ 46.]
[ 20.]


In [7]:
# 活性化関数の確認
x = Variable(np.array([-1], dtype=np.float32))
# sin関数
print(F.sin(x).data)
# シグモイド関数
print(F.sigmoid(x).data)

[-0.84147096]
[ 0.2689414]


In [8]:
x = Variable(np.array([-1, 0, 1], dtype=np.float32))
z = F.sin(x)
z.grad = np.ones(3, dtype=np.float32)
z.backward()
print(x.grad)

[ 0.54030228  1.          0.54030228]


In [11]:
# 3 * 4
h = L.Linear(3, 4)
# 重みの初期値は適当な値が入る
print(h.W.data)

print(h.b.data)

[[-1.01418269 -0.47092569 -0.18234934]
 [ 0.65506345 -1.03648019  0.19952714]
 [ 0.24183242 -0.35688877  0.2783677 ]
 [-0.20961381  0.03233764 -0.18798847]]
[ 0.  0.  0.  0.]


In [12]:
x = Variable(np.array(range(6)).astype(np.float32).reshape(2, 3))
print(x.data)
y = h(x)
# hに当てはめての計算結果表示
print("実際の値")
print(y.data)

w = h.W.data
x0 = x.data
print("計算値チェック")
print(x0.dot(w.T) + h.b.data)

[[ 0.  1.  2.]
 [ 3.  4.  5.]]
実際の値
[[-0.83562434 -0.6374259   0.19984663 -0.34363931]
 [-5.83799744 -1.18309462  0.68978065 -1.43943334]]
計算値チェック
[[-0.83562434 -0.6374259   0.19984663 -0.34363931]
 [-5.83799744 -1.18309462  0.68978065 -1.43943334]]


In [14]:
# Chainクラス
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4, 3),
            l2 = L.Linear(3, 3)
        )
    
    def __call__(self, x,y):
        fv = self.fwd(x,y)
        loss = F.mean_squared_error(fv, y)
        return loss
    
    def fwd(self, x, y):
        return F.sigmoid(self.l1(x))
    

In [39]:
# 3* 4 行列
h = L.Linear(4, 3)
x = Variable(np.array(range(12)).astype(np.float32).reshape(3, 4))
y = h(x)
print(h.W.data)
print(x.data)
print(y.data)

[[  2.73467302e-01   7.20895946e-01   3.28617066e-01   4.70501967e-02]
 [  9.39238906e-01   2.95145720e-01   8.35358873e-02   1.95321583e-04]
 [  1.67628661e-01   4.01083171e-01  -2.41401061e-01  -7.62534142e-02]]
[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]]
[[  1.51928067   0.46280345  -0.31047922]
 [  6.999403     5.73526669   0.69375026]
 [ 12.47952461  11.00773048   1.69797993]]


In [93]:
model = MyChain() # モデルの生成
optimizer = optimizers.SGD() # 最適化のアルゴリズムの選択
optimizer.setup(model) # アルゴリズムにモデルをセット

In [94]:
model.zerograds() # 勾配の初期化
loss = model(x, y)  #順方向に計算して誤差を算出
print(loss.data)
for i in range(15):
    loss = model(x, y)  #順方向に計算して誤差を算出
    loss.backward() # 誤差逆伝搬、　勾配の計算
    optimizer.update() # パラメータを更新
print(loss.data)

32.74905776977539
32.69089126586914


In [95]:
# hに当てはめての計算結果表示
print("実際の値")
print(y.data)

w1 = model.l1.W.data
b1 = model.l1.b.data
w2 = model.l2.W.data
b2 = model.l2.b.data

x0 = x.data
x1 = x0.dot(w1.T) + b1.data
x2 = x1.dot(w2.T) + b2.data

print("計算値チェック")
print(x0)
print(x1)
print(x2)

実際の値
[[  1.51928067   0.46280345  -0.31047922]
 [  6.999403     5.73526669   0.69375026]
 [ 12.47952461  11.00773048   1.69797993]]
計算値チェック
[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]]
[[  1.85242999   1.21912575  -2.08572459]
 [  7.5033741    5.61429977  -2.32977486]
 [ 13.15431881  10.0094738   -2.57382488]]
[[ 0.84466881  1.66653788  2.05560446]
 [ 3.63455749  3.61122322  3.07467985]
 [ 6.42444658  5.5559082   4.09375477]]


In [145]:
# Chainクラス
class MyChain2(Chain):
    def __init__(self):
        super(MyChain2, self).__init__(
            l1 = L.Linear(4, 1),
            l2 = L.Linear(1, 1)
        )
    
    def __call__(self, x,y):
        fv = self.fwd(x,y)
        loss = F.mean_squared_error(fv, y)
        return loss
    
    def fwd(self, x, y):
        return F.sigmoid(self.l1(x))
    

In [146]:
# 3* 4 行列
h = L.Linear(4, 1)
x = Variable(np.array(range(16)).astype(np.float32).reshape(4, 4))
y = h(x)
print(h.W.data)
print(x.data)
print(y.data)

[[ 0.09960782 -0.20798427  0.40322188 -0.86007929]]
[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]]
[[-1.98177838]
 [-4.24271393]
 [-6.50364971]
 [-8.76458549]]


In [151]:
model = MyChain2() # モデルの生成
optimizer = optimizers.SGD() # 最適化のアルゴリズムの選択
optimizer.setup(model) # アルゴリズムにモデルをセット

In [152]:
model.zerograds() # 勾配の初期化
loss = model(x, y)  #順方向に計算して誤差を算出
print(loss.data)
for i in range(10):
    loss = model(x, y)  #順方向に計算して誤差を算出
    loss.backward() # 誤差逆伝搬、　勾配の計算
    optimizer.update() # パラメータを更新
print(loss.data)

35.285057067871094
35.281578063964844


In [153]:
# hに当てはめての計算結果表示
print("実際の値")
print(y.data)

w1 = model.l1.W.data
b1 = model.l1.b.data
w2 = model.l2.W.data
b2 = model.l2.b.data

x0 = x.data
x1 = x0.dot(w1.T) + b1.data
x2 = x1.dot(w2.T) + b2.data

print("計算値チェック")
print("input", x0)
print("l1", x1)
print("l2", x2)

実際の値
[[-1.98177838]
 [-4.24271393]
 [-6.50364971]
 [-8.76458549]]
計算値チェック
input [[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]]
l1 [[ -3.88896322]
 [-10.63428307]
 [-17.37960243]
 [-24.12492371]]
l2 [[  2.64205647]
 [  7.22464466]
 [ 11.8072319 ]
 [ 16.3898201 ]]
